In [31]:
import os
import pandas as pd
import numpy as np
import gc

In [32]:
folderPath = "D:\Competitions\CorporationGrocerySalesPrediction"

In [207]:
fileName = "train.csv"
filePath = os.path.join(folderPath,fileName)
types = {
    'item_nbr':np.uint8,
    'store_nbr':np.uint8,
    'unit_sales':np.float16,
    'onpromotion':'object'
}
data = pd.read_csv(filePath
                   ,usecols=['store_nbr','date','item_nbr','unit_sales','onpromotion']
                   ,dtype=types
                  )

In [208]:
data["onpromotion"].fillna(False,inplace=True)

In [209]:
data["onpromotion"] = data["onpromotion"].astype(np.bool)

In [210]:
onpromotion={True : 1,False: 0}
data["onpromotion"]=data["onpromotion"].map(onpromotion)
data["onpromotion"] = data["onpromotion"].astype(np.int8)

In [211]:
#data["date"] =  pd.to_datetime(data["date"])

In [212]:
#data = data.loc[(data.unit_sales>=0)]
data["unit_sales"].fillna(0,inplace=True)

In [134]:
data.reset_index(inplace=True)
data.drop("index",axis=1,inplace=True)

In [213]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
date           object
store_nbr      uint8
item_nbr       uint8
unit_sales     float16
onpromotion    int8
dtypes: float16(1), int8(1), object(1), uint8(2)
memory usage: 1.5+ GB


,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,241,7.0,0
1,2013-01-01,25,102,1.0,0
2,2013-01-01,25,103,2.0,0
3,2013-01-01,25,47,1.0,0
4,2013-01-01,25,157,1.0,0


## Using Mean

In [214]:
data.drop("date",axis = 1,inplace=True)

In [215]:
data["unit_sales"] = data["unit_sales"].apply(np.log1p)

In [216]:
data = data.groupby(['item_nbr','store_nbr','onpromotion']).agg({'unit_sales':'mean'})

In [217]:
data["unit_sales"] = data["unit_sales"].replace([np.inf,-np.inf],np.nan)

In [218]:
data["unit_sales"].fillna(0,inplace=True)

In [219]:
data[np.isfinite(data["unit_sales"]) == False]

,,,unit_sales
item_nbr,store_nbr,onpromotion,


In [220]:
data["unit_sales"] = data["unit_sales"].apply(np.expm1)

In [221]:
data['unit_sales'] = round(data['unit_sales']).astype(np.uint32)

In [222]:
#reading test file
fileName = "test.csv"
filePath = os.path.join(folderPath,fileName)
test = pd.read_csv(filePath,usecols=['id','store_nbr','item_nbr','onpromotion'],index_col=['item_nbr','store_nbr','onpromotion'])

C:\Users\Inspiron\Anaconda3\envs\tensorflow\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [223]:
test.join(data,how='left').fillna(0).to_csv(os.path.join(folderPath,'submission19_11_2017_5Test.csv')
                                            ,float_format="%.2f"
                                            ,index=None)

In [224]:
del(test)

In [225]:
del(data)

In [226]:
gc.collect()

7